In [3]:
# start the spark server

from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('soccer_analytics').getOrCreate()

In [4]:

events_df = spark.read.csv("../raw/data/Dataset/events_England.csv", header=True)
events_df.show(truncate=False)

+-------+------------+----------------------------+--------+----------------------------------------+-------+---------+------+-----------+------------------+----------+---------+------------+----------+----------+----------+----------+
|eventId|subEventName|tags                        |playerId|positions                               |matchId|eventName|teamId|matchPeriod|eventSec          |subEventId|id       |tagsList    |pos_orig_y|pos_orig_x|pos_dest_y|pos_dest_x|
+-------+------------+----------------------------+--------+----------------------------------------+-------+---------+------+-----------+------------------+----------+---------+------------+----------+----------+----------+----------+
|8      |Simple pass |[{'id': 1801}]              |25413   |[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]|2499719|Pass     |1609  |1H         |2.7586489999999912|85.0      |177959171|[1801]      |49        |49        |78        |31        |
|8      |High pass   |[{'id': 1801}]              |37022

In [5]:
from pyspark.sql.functions import explode

players_df = spark.read.csv("../data/raw/Dataset/players.csv", header=True)
players_df.show(truncate=False)

+------------------------------------------------------------------------------+------+--------------------+----------+---------------------------+-------------+----------+------+-----------------------------------------------------+------------------------------------------------------------------------------+------+-----+-------------------------+---------------------+
|passportArea                                                                  |weight|firstName           |middleName|lastName                   |currentTeamId|birthDate |height|role                                                 |birthArea                                                                     |wyId  |foot |shortName                |currentNationalTeamId|
+------------------------------------------------------------------------------+------+--------------------+----------+---------------------------+-------------+----------+------+-----------------------------------------------------+-------------------

In [6]:
# left join the events and player dataframe

from pyspark.sql.functions import col, concat, count, lit

combine_df = events_df.join(players_df, on=events_df.playerId == players_df.wyId, how="left") \
                .where((col("subEventName") == "Penalty") & (col("eventName") == "Free Kick"))
combine_df.show(truncate=False)

+-------+------------+------------------------------------------------------+--------+----------------------------------------+-------+---------+------+-----------+------------------+----------+---------+----------------------+----------+----------+----------+----------+--------------------------------------------------------------------------+------+----------------+----------+------------------------+-------------+----------+------+-----------------------------------------------------+--------------------------------------------------------------------------+------+-----+-------------------+---------------------+
|eventId|subEventName|tags                                                  |playerId|positions                               |matchId|eventName|teamId|matchPeriod|eventSec          |subEventId|id       |tagsList              |pos_orig_y|pos_orig_x|pos_dest_y|pos_dest_x|passportArea                                                              |weight|firstName       |middleN

In [7]:
# Select required column and create new columns
from pyspark.sql.functions import from_json, col, size, try_divide

selected_df = combine_df.select(col("playerId"), concat(col("firstName"), lit(" "), col("lastName")).alias("playerName"), size(from_json(col("tags"), "array<map<string,string>>")).alias("numPKGoals"))
selected_df = selected_df.groupBy("playerId", "playerName", "numPKGoals").agg(count("playerId").alias("numPKAtt"))
selected_df = selected_df.withColumn("PKSuccessRate", try_divide(left=col("numPKGoals"),right=col("numPKAtt")))
selected_df.filter(col("numPkAtt") >=5).show(truncate=False)

+--------+---------------------+----------+--------+------------------+
|playerId|playerName           |numPKGoals|numPKAtt|PKSuccessRate     |
+--------+---------------------+----------+--------+------------------+
|12829   |Jamie Vardy          |4         |5       |0.8               |
|127537  |Luka Milivojevi\u0107|4         |7       |0.5714285714285714|
+--------+---------------------+----------+--------+------------------+



In [8]:
spark.stop()